In [56]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime
from nltk.corpus import stopwords
import gensim
import smart_open
from string import punctuation
stop_words = stopwords.words('english')
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Lambda
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint

In [39]:
df = pd.read_csv('train.csv')

In [40]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [19]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('C:\\Users\\Suraj\\GoogleNews-vectors-negative300.bin.gz', binary=True)

In [41]:
word2vec.vocab

{'</s>': <gensim.models.keyedvectors.Vocab at 0x1826fc64748>,
 'in': <gensim.models.keyedvectors.Vocab at 0x1826fc64f98>,
 'for': <gensim.models.keyedvectors.Vocab at 0x1826fc64c88>,
 'that': <gensim.models.keyedvectors.Vocab at 0x1826fc645c0>,
 'is': <gensim.models.keyedvectors.Vocab at 0x1826fc64e48>,
 'on': <gensim.models.keyedvectors.Vocab at 0x1826fc649e8>,
 '##': <gensim.models.keyedvectors.Vocab at 0x1826fc64320>,
 'The': <gensim.models.keyedvectors.Vocab at 0x1826fc644a8>,
 'with': <gensim.models.keyedvectors.Vocab at 0x1826fc64d68>,
 'said': <gensim.models.keyedvectors.Vocab at 0x1826fc64128>,
 'was': <gensim.models.keyedvectors.Vocab at 0x18168be8ac8>,
 'the': <gensim.models.keyedvectors.Vocab at 0x18168be8080>,
 'at': <gensim.models.keyedvectors.Vocab at 0x18168be80f0>,
 'not': <gensim.models.keyedvectors.Vocab at 0x18168be8160>,
 'as': <gensim.models.keyedvectors.Vocab at 0x18168be81d0>,
 'it': <gensim.models.keyedvectors.Vocab at 0x18168be8240>,
 'be': <gensim.models.keyed

In [42]:
#Cleaning


SPECIAL_TOKENS = {
    'quoted': 'quoted_item',
    'non-ascii': 'non_ascii_word',
    'undefined': 'something'
}


def clean(text, stem_words=True):

    
    if pd.isnull(text):
        return ''

    if type(text) != str or text=='':
        return ''
    ps = PorterStemmer()
    
    text = re.sub("\'s", " ", text) 
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub("(\d+)(kK)", " \g<1>000 ", text)
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    
    text = text.split()
    
    return text


In [43]:
df['question1'] = df['question1'].apply(clean)
df['question2'] = df['question2'].apply(clean)

In [44]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[What, is, the, step, by, step, guide, to, inv...","[What, is, the, step, by, step, guide, to, inv...",0
1,1,3,4,"[What, is, the, story, of, Kohinoor, (Koh-i-No...","[What, would, happen, if, the, Indian, governm...",0
2,2,5,6,"[How, can, I, increase, the, speed, of, my, in...","[How, can, Internet, speed, be, increased, by,...",0
3,3,7,8,"[Why, am, I, mentally, very, lonely?, How, can...","[Find, the, remainder, when, [math]23^{24}[/ma...",0
4,4,9,10,"[Which, one, dissolve, in, water, quikly, suga...","[Which, fish, would, survive, in, salt, water?]",0


In [46]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']
questions_cols = ['question1', 'question2']

In [47]:
len(inverse_vocabulary)

1

In [48]:
for index,row in df.iterrows():
    # Iterate through the text of both questions of the row
    for question in questions_cols:
        q2n = []  # q2n -> question numbers representation
        for word in row[question]:
            # Check for unwanted words
            if word in stop_words and word not in word2vec.vocab:
                continue
            if word not in vocabulary:
                vocabulary[word] = len(inverse_vocabulary)
                q2n.append(len(inverse_vocabulary))
                inverse_vocabulary.append(word)
            else:
                q2n.append(vocabulary[word])
        df.at[index, question] = q2n      

In [49]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 12]",0
1,1,3,4,"[1, 2, 3, 13, 14, 15, 16]","[1, 17, 18, 19, 3, 20, 21, 22, 3, 14, 15, 23, 24]",0
2,2,5,6,"[25, 26, 27, 28, 3, 29, 30, 31, 32, 33, 34, 35]","[25, 26, 36, 29, 37, 38, 5, 39, 40, 41]",0
3,3,7,8,"[42, 43, 27, 44, 45, 46, 25, 26, 27, 47, 48]","[49, 3, 50, 51, 52, 2, 53, 5, 54]",0
4,4,9,10,"[55, 56, 57, 8, 58, 59, 60, 61, 62, 63, 64, 65]","[55, 66, 17, 67, 8, 68, 69]",0


In [50]:
vocabulary

{'What': 1,
 'is': 2,
 'the': 3,
 'step': 4,
 'by': 5,
 'guide': 6,
 'invest': 7,
 'in': 8,
 'share': 9,
 'market': 10,
 'india?': 11,
 'market?': 12,
 'story': 13,
 'Kohinoor': 14,
 '(Koh-i-Noor)': 15,
 'Diamond?': 16,
 'would': 17,
 'happen': 18,
 'if': 19,
 'Indian': 20,
 'government': 21,
 'stole': 22,
 'diamond': 23,
 'back?': 24,
 'How': 25,
 'can': 26,
 'I': 27,
 'increase': 28,
 'speed': 29,
 'my': 30,
 'internet': 31,
 'connection': 32,
 'while': 33,
 'using': 34,
 'VPN?': 35,
 'Internet': 36,
 'be': 37,
 'increased': 38,
 'hacking': 39,
 'through': 40,
 'DNS?': 41,
 'Why': 42,
 'am': 43,
 'mentally': 44,
 'very': 45,
 'lonely?': 46,
 'solve': 47,
 'it?': 48,
 'Find': 49,
 'remainder': 50,
 'when': 51,
 '[math]23^{24}[/math]': 52,
 'divided': 53,
 '2423?': 54,
 'Which': 55,
 'one': 56,
 'dissolve': 57,
 'water': 58,
 'quikly': 59,
 'sugar,': 60,
 'salt,': 61,
 'methane': 62,
 'carbon': 63,
 'di': 64,
 'oxide?': 65,
 'fish': 66,
 'survive': 67,
 'salt': 68,
 'water?': 69,
 'Ast

In [51]:
len(vocabulary)

227271

In [52]:
df.drop(['id','qid1','qid2'],axis = 1,inplace = True)

In [53]:
df.head()

,question1,question2,is_duplicate
0,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 12]",0
1,"[1, 2, 3, 13, 14, 15, 16]","[1, 17, 18, 19, 3, 20, 21, 22, 3, 14, 15, 23, 24]",0
2,"[25, 26, 27, 28, 3, 29, 30, 31, 32, 33, 34, 35]","[25, 26, 36, 29, 37, 38, 5, 39, 40, 41]",0
3,"[42, 43, 27, 44, 45, 46, 25, 26, 27, 47, 48]","[49, 3, 50, 51, 52, 2, 53, 5, 54]",0
4,"[55, 56, 57, 8, 58, 59, 60, 61, 62, 63, 64, 65]","[55, 66, 17, 67, 8, 68, 69]",0


In [ ]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)
embeddings[0] = 0
# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

In [96]:
len(embeddings)

227272

In [57]:
X = df.iloc[:,0:2]
y = df['is_duplicate']

In [70]:
X.shape,y.shape

((404290, 2), (404290,))

In [69]:
X.head()

,question1,question2
0,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 12]"
1,"[1, 2, 3, 13, 14, 15, 16]","[1, 17, 18, 19, 3, 20, 21, 22, 3, 14, 15, 23, 24]"
2,"[25, 26, 27, 28, 3, 29, 30, 31, 32, 33, 34, 35]","[25, 26, 36, 29, 37, 38, 5, 39, 40, 41]"
3,"[42, 43, 27, 44, 45, 46, 25, 26, 27, 47, 48]","[49, 3, 50, 51, 52, 2, 53, 5, 54]"
4,"[55, 56, 57, 8, 58, 59, 60, 61, 62, 63, 64, 65]","[55, 66, 17, 67, 8, 68, 69]"


In [59]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: is_duplicate, dtype: int64

#### Prepare training and validation data

In [63]:
Xt,X_test,yt,y_test = train_test_split(X,y,stratify=y,test_size=0.02,shuffle = True)

In [64]:
Xt.shape,yt.shape

((396204, 2), (396204,))

In [65]:
X_test.shape,y_test.shape

((8086, 2), (8086,))

In [66]:
validation_size = 40000
X_train, X_validation, Y_train, Y_validation = train_test_split(Xt, yt, test_size=validation_size)

In [67]:
X_train.shape,Y_train.shape

((356204, 2), (356204,))

In [68]:
X_validation.shape,Y_validation.shape

((40000, 2), (40000,))

In [71]:
max_seq_length = max(X.question1.map(lambda x: len(x)).max(),
                     X.question2.map(lambda x: len(x)).max(),
                     )

In [72]:
max_seq_length

214

In [73]:
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': X_test.question1, 'right': X_test.question2}

Y_train = Y_train.values
Y_validation = Y_validation.values
Y_test = y_test.values

In [76]:
for dataset,side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)    

In [80]:
X_train['left'][0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [81]:
X_train['right'][0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [89]:
X_train

{'left': array([[     0,      0,      0, ...,   1405,  10137,   5046],
        [     0,      0,      0, ...,   8900,   2525,   3521],
        [     0,      0,      0, ...,   6046,  12909,    905],
        ...,
        [     0,      0,      0, ..., 155256,  58500,   5406],
        [     0,      0,      0, ...,   8946,      8,   7870],
        [     0,      0,      0, ...,    219,    220,  11470]]),
 'right': array([[     0,      0,      0, ...,      3,   1405,   1511],
        [     0,      0,      0, ...,   2525,      8,   1380],
        [     0,      0,      0, ...,   6046,  12909,    905],
        ...,
        [     0,      0,      0, ...,      2, 155256,  31945],
        [     0,      0,      0, ...,   3170,      8,   8384],
        [     0,      0,      0, ...,   5566,      8,   1069]])}

In [92]:
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [98]:
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [100]:
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 25
# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, epochs=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

Epoch 1/25
5566/5566 [==============================] - 1081s 194ms/step - loss: 0.2907 - accuracy: 0.6531 - val_loss: 0.2879 - val_accuracy: 0.6567
Epoch 2/25
5566/5566 [==============================] - 1062s 191ms/step - loss: 0.2898 - accuracy: 0.6541 - val_loss: 0.2870 - val_accuracy: 0.6577
Epoch 3/25
5566/5566 [==============================] - 1078s 194ms/step - loss: 0.2890 - accuracy: 0.6552 - val_loss: 0.2863 - val_accuracy: 0.6589
Epoch 4/25
5566/5566 [==============================] - 1068s 192ms/step - loss: 0.2882 - accuracy: 0.6563 - val_loss: 0.2856 - val_accuracy: 0.6598
Epoch 5/25
5566/5566 [==============================] - 1072s 193ms/step - loss: 0.2875 - accuracy: 0.6573 - val_loss: 0.2849 - val_accuracy: 0.6607
Epoch 6/25
5566/5566 [==============================] - 1032s 185ms/step - loss: 0.2868 - accuracy: 0.6581 - val_loss: 0.2843 - val_accuracy: 0.6618
Epoch 7/25
5566/5566 [==============================] - 1037s 186ms/step - loss: 0.2862 - accuracy: 0.6592

KeyboardInterrupt: 